In [1]:
# Google Colaboratoryを使用する場合は、下記のコメントアウトを外し、実行してください

In [2]:
# # 下記セルを実行すると、authorization codeの入力を求められます。
# # 出力されたリンク先をクリックし、Googleアカウントにログインし、
# # authorization codeをコピーし、貼り付けをおこなってください。
# import os
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# working_dir = 'MLSys_100Knocks' #　※※自分で作成したフォルダパスが異なる場合こちらを変更してください。※※
# path = f'/content/drive/MyDrive/{working_dir}/本章/6章'
# os.chdir(path)

# ６章 機械学習のためのデータ加工をする１０本ノック



### ノック５１：データ加工の下準備をしよう

In [4]:
import os
data_dir = "data"
input_dir = os.path.join(data_dir, "0_input")
output_dir = os.path.join(data_dir, "1_output")
master_dir = os.path.join(data_dir, "99_master")
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(master_dir, exist_ok=True)

##### ＊＊必ずデータの配置をおこなってください。＊＊

#### MacやGoogle Colaboratory等、環境によっては、ファイル取得時の順番が年月順にならないことがあります。
#### その場合、tbl_order_paths = sorted(tbl_order_paths)を入れてください。

In [5]:
import glob
tbl_order_file = os.path.join(input_dir, "tbl_order_*.csv")
tbl_order_paths = glob.glob(tbl_order_file)
tbl_order_paths

['data/0_input/tbl_order_201905.csv',
 'data/0_input/tbl_order_202002.csv',
 'data/0_input/tbl_order_201904.csv',
 'data/0_input/tbl_order_201907.csv',
 'data/0_input/tbl_order_201909.csv',
 'data/0_input/tbl_order_201910.csv',
 'data/0_input/tbl_order_202003.csv',
 'data/0_input/tbl_order_201911.csv',
 'data/0_input/tbl_order_201908.csv',
 'data/0_input/tbl_order_201906.csv',
 'data/0_input/tbl_order_201912.csv',
 'data/0_input/tbl_order_202001.csv']

### ノック５２：データの読み込みを行い加工の方向性を検討しよう

In [6]:
import pandas as pd
m_area_file = "m_area.csv"
m_store_file = "m_store.csv"
m_area = pd.read_csv(os.path.join(master_dir, m_area_file))
m_store = pd.read_csv(os.path.join(master_dir, m_store_file))
m_area.head(3)

,area_cd,wide_area,narrow_area
0,TK,東京,東京
1,KN,神奈川,神奈川
2,CH,千葉,千葉


In [7]:
tbl_order_path = tbl_order_paths[0]
print(f'読み込みデータ：{tbl_order_path}')
order_data = pd.read_csv(tbl_order_path)
print(f'データ件数：{len(order_data)}')
order_data.head(3)

読み込みデータ：data/0_input/tbl_order_201905.csv
データ件数：241041


,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status
0,24271788,136,C09404153,30,27906557,2019-05-01 11:00:00,2019-05-01 11:55:00,0,4744,2
1,90187608,22,C05404908,81,36668327,2019-05-01 11:00:00,2019-05-01 11:21:00,0,2328,2
2,62067185,121,C72617833,65,59085804,2019-05-01 11:00:00,2019-05-01 11:50:00,0,2877,2


### ノック５３：１か月分のデータの基本的なデータ加工を実施しよう

In [8]:
order_data = order_data.loc[order_data["store_id"]!=999]

order_data = pd.merge(order_data, m_store, on="store_id", how="left")
order_data = pd.merge(order_data, m_area, on="area_cd", how="left")

order_data.loc[order_data["takeout_flag"]==0, "takeout_name"] = "デリバリー"
order_data.loc[order_data["takeout_flag"]==1, "takeout_name"] = "お持ち帰り"

order_data.loc[order_data["status"]==0, "status_name"] = "受付"
order_data.loc[order_data["status"]==1, "status_name"] = "お支払済"
order_data.loc[order_data["status"]==2, "status_name"] = "お渡し済"
order_data.loc[order_data["status"]==9, "status_name"] = "キャンセル"
order_data.head(3)

,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status,store_name,area_cd,wide_area,narrow_area,takeout_name,status_name
0,24271788,136,C09404153,30,27906557,2019-05-01 11:00:00,2019-05-01 11:55:00,0,4744,2,木更津店,CH,千葉,千葉,デリバリー,お渡し済
1,90187608,22,C05404908,81,36668327,2019-05-01 11:00:00,2019-05-01 11:21:00,0,2328,2,国立店,TK,東京,東京,デリバリー,お渡し済
2,62067185,121,C72617833,65,59085804,2019-05-01 11:00:00,2019-05-01 11:50:00,0,2877,2,港南店,KN,神奈川,神奈川,デリバリー,お渡し済


In [9]:
order_data.isna().sum()

order_id             0
store_id             0
customer_id          0
coupon_cd            0
sales_detail_id      0
order_accept_date    0
delivered_date       0
takeout_flag         0
total_amount         0
status               0
store_name           0
area_cd              0
wide_area            0
narrow_area          0
takeout_name         0
status_name          0
dtype: int64

### ノック５４：機械学習に使用する変数を作成しよう

In [11]:
def calc_delta(t):
    t1, t2 = t
    delta = t2 - t1
    return delta.total_seconds()/60

order_data.loc[:, "order_accept_datetime"] = pd.to_datetime(order_data["order_accept_date"])
order_data.loc[:, "delivered_datetime"] = pd.to_datetime(order_data["delivered_date"])
order_data.loc[:, "delta"] = order_data[["order_accept_datetime", "delivered_datetime"]].apply(calc_delta, axis=1)
order_data.head(3)

,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status,store_name,area_cd,wide_area,narrow_area,takeout_name,status_name,order_accept_datetime,delivered_datetime,delta
0,24271788,136,C09404153,30,27906557,2019-05-01 11:00:00,2019-05-01 11:55:00,0,4744,2,木更津店,CH,千葉,千葉,デリバリー,お渡し済,2019-05-01 11:00:00,2019-05-01 11:55:00,55.0
1,90187608,22,C05404908,81,36668327,2019-05-01 11:00:00,2019-05-01 11:21:00,0,2328,2,国立店,TK,東京,東京,デリバリー,お渡し済,2019-05-01 11:00:00,2019-05-01 11:21:00,21.0
2,62067185,121,C72617833,65,59085804,2019-05-01 11:00:00,2019-05-01 11:50:00,0,2877,2,港南店,KN,神奈川,神奈川,デリバリー,お渡し済,2019-05-01 11:00:00,2019-05-01 11:50:00,50.0


In [12]:
order_data.loc[:, "order_accept_hour"] = order_data["order_accept_datetime"].dt.hour
order_data.loc[:, "order_accept_weekday"] = order_data["order_accept_datetime"].dt.weekday
order_data.loc[order_data["order_accept_weekday"]>=5, "weekday_info"] = "休日"
order_data.loc[order_data["order_accept_weekday"]<5, "weekday_info"] = "平日"
order_data.head(3)


,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status,...,wide_area,narrow_area,takeout_name,status_name,order_accept_datetime,delivered_datetime,delta,order_accept_hour,order_accept_weekday,weekday_info
0,24271788,136,C09404153,30,27906557,2019-05-01 11:00:00,2019-05-01 11:55:00,0,4744,2,...,千葉,千葉,デリバリー,お渡し済,2019-05-01 11:00:00,2019-05-01 11:55:00,55.0,11,2,平日
1,90187608,22,C05404908,81,36668327,2019-05-01 11:00:00,2019-05-01 11:21:00,0,2328,2,...,東京,東京,デリバリー,お渡し済,2019-05-01 11:00:00,2019-05-01 11:21:00,21.0,11,2,平日
2,62067185,121,C72617833,65,59085804,2019-05-01 11:00:00,2019-05-01 11:50:00,0,2877,2,...,神奈川,神奈川,デリバリー,お渡し済,2019-05-01 11:00:00,2019-05-01 11:50:00,50.0,11,2,平日


### ノック５５：店舗単位に集計して変数を作成しよう

### ノック５６：データの加工と店舗別集計を関数で実行しよう

### ノック５７：全データの読み込みとデータ加工をやってみよう

### ノック５８：目的変数を作成しよう

### ノック５９：説明変数と目的変数を紐づけて機械学習用のデータを仕上げよう

### ノック６０：機械学習用データの確認を行い出力しよう